In [1]:
#Heart Failure Readmissions Performance Analysis
#Colorado Hospitals, 2020 - 2023

In [2]:
#Business Problem
#Hospital readmissions for Heart Failure are a major quality and cost concern in U.S. healthcare. The Centers for Medicare & Medicaid Services (CMS) publicly reports hospital performance using Excess Readmission Ratio (ERR).
#Healthcare leaders must:
#•	Identify hospitals with performance variation
#•	Detect improvement opportunities
#•	Understand reporting participation gaps
#•	Prioritize quality improvement initiatives

#Project Objectives
#This project aims to:
#1.	Clean and prepare CMS hospital readmissions data using Python
#2.	Standardize and anonymize hospital identifiers
#3.	Create Tableau-ready performance metrics
#4.	Develop interactive dashboards to evaluate:#
#o	Performance distribution
#o	Reporting participation
#o	Performance tiers
#o	Volume vs. quality relationships

#Tools & Technologies
#•	Python (Pandas, NumPy, Path)
#•	Jupyter Notebook
#•	Tableau Public
#•	CMS Hospital Compare Dataset

#Data Source
#Dataset obtained from: https://data.cms.gov/provider-data/dataset/9n3s-kdb3, stored locally as .csv file.
#Key fields used:
#•	Hospital Code
#•	Hospital Name
#•	State
#•	Reporting Status
#•	Excess Readmission Ratio
#•	Predicted Readmissions
#•	Expected Readmissions


In [3]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
# Step 2: Load Dataset and validate using Path for clarity, safety, and scalability
DATA_DIR = Path(".")
RAW_FILE = "raw_hf.csv"

csv_path = DATA_DIR / RAW_FILE
df_raw = pd.read_csv(csv_path, dtype=str)
df_raw.head()

,Facility Name,Facility ID,State,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date
0,BANNER NORTH COLORADO MEDICAL CENTER,060001,CO,READM-30-HF-HRRP,196,NaN,0.9433,17.3529,18.3964,30,07/01/2020,06/30/2023
1,LONGMONT UNITED HOSPITAL,060003,CO,READM-30-HF-HRRP,87,NaN,1.0015,19.4159,19.3858,17,07/01/2020,06/30/2023
2,INTERMOUNTAIN HEALTH PLATTE VALLEY HOSPITAL,060004,CO,READM-30-HF-HRRP,NaN,NaN,0.9784,17.9847,18.3811,Too Few to Report,07/01/2020,06/30/2023
3,MONTROSE REGIONAL HEALTH,060006,CO,READM-30-HF-HRRP,NaN,NaN,0.9384,16.0681,17.1226,Too Few to Report,07/01/2020,06/30/2023
4,SAN LUIS VALLEY REGIONAL MEDICAL CENTER,060008,CO,READM-30-HF-HRRP,NaN,1,NaN,NaN,NaN,NaN,07/01/2020,06/30/2023


In [5]:
# Step 3: Data Exploration to understand structure, detect missing values and identify data issues 

In [6]:
df_raw.info()
df_raw.describe()
df_raw.isnull().sum()
df_raw.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Facility Name               51 non-null     object
 1   Facility ID                 51 non-null     object
 2   State                       51 non-null     object
 3   Measure Name                51 non-null     object
 4   Number of Discharges        34 non-null     object
 5   Footnote                    10 non-null     object
 6   Excess Readmission Ratio    41 non-null     object
 7   Predicted Readmission Rate  41 non-null     object
 8   Expected Readmission Rate   41 non-null     object
 9   Number of Readmissions      41 non-null     object
 10  Start Date                  51 non-null     object
 11  End Date                    51 non-null     object
dtypes: object(12)
memory usage: 4.9+ KB


Index(['Facility Name', 'Facility ID', 'State', 'Measure Name',
       'Number of Discharges', 'Footnote', 'Excess Readmission Ratio',
       'Predicted Readmission Rate', 'Expected Readmission Rate',
       'Number of Readmissions', 'Start Date', 'End Date'],
      dtype='object')

In [7]:
# Step 4: Data Backup (for provenance) then Start Data Cleaning
df = df_raw.copy()

In [8]:
# Filtering the dataset to the analytic scope of 30-day Readmissions Heart Failure in Colorado regardless of the upstream filtering at the CMS site done to reduce file size
# Preserve the filtered dataset as df_hf
df_hf = df[
    (df['State'] == 'CO') &
    (df['Measure Name'] == 'READM-30-HF-HRRP')
].copy()

In [9]:
# Raw CMS column names were normalized to concise, snake_case variables to improve readability, consistency, and reuse across analyses.
df_hf.columns = (
    df_hf.columns
      .str.strip()
      .str.replace(r"[^\w]+", "_", regex=True)
      .str.lower()
)

In [10]:
# Create Column MAP to preserve the validity of the column labels used in the analysis for stability, even if the CMS data label(s) subsequently change.


In [11]:
COLS = {
    "facility_name": "hospital_name",
    "facility_id": "hospital_code",
    "state": "state",
    "measure_name": "measure",
    "number_of_discharges": "discharges",
    "number_of_readmissions": "readmissions",
    "excess_readmission_ratio": "excess_ratio",
    "predicted_readmission_rate": "predicted_rate",
    "expected_readmission_rate": "expected_rate",
    "start_date": "start_date",
    "end_date": "end_date"
}

In [12]:
# Adopt the above analytic column labels
df_hf = df_hf.rename(columns=COLS)

In [13]:
# CMS suppresses readmission metrics for hospitals with insufficient case volume or reporting limitations. These hospitals were identified and excluded from comparative performance analysis but retained for transparency.
# I created reporting status flag using the "readmissions" column. The latter was normalized prior to classification to account for inconsistent string formatting, trailing spaces,and missing values present in the raw data.


In [14]:
df_hf['readmissions'] = df_hf['readmissions'].astype(str).str.strip().str.lower()

In [15]:
def reporting_status(x):
    if 'too few' in x:
        return 'Too Few Cases'
    elif x in ['n/a', 'na', 'nan', 'not available']:
        return 'Not Available'
    else:
        return 'Reported'

In [16]:
df_hf['reporting_status'] = df_hf['readmissions'].apply(reporting_status)

In [17]:
# Verify that the above function correctly applied to the "readmissions" column
df_hf['reporting_status'].unique()

array(['Reported', 'Too Few Cases', 'Not Available'], dtype=object)

In [18]:
df_hf['reporting_status'].value_counts()

reporting_status
Reported         34
Not Available    10
Too Few Cases     7
Name: count, dtype: int64

In [19]:
# Anonymization of Hospitals: the goal is to remove the hospital names and identifiable facility_id, derive a hospital code, (H_XX), for each hospital from their current hospital_id, and create a column for the hospital_code in the dataframe.
# 1) Rename the anonymized dataframe, "df_anon and preserve df_hf for provenance.
df_anon = df_hf.copy()

In [20]:
df_anon = df_anon.drop(columns=["hospital_name"])

In [21]:
df_anon = df_anon.rename(columns={'hospital_id': 'hospital_code'})

In [22]:
hospital_map = {
    old_id: f"H_{str(i+1).zfill(2)}"
    for i, old_id in enumerate(df_anon['hospital_code'].unique())
}


In [23]:
df_anon['hospital_code'] = df_anon['hospital_code'].map(hospital_map)

In [24]:
# Numeric conversion: This following steps converts numerical columns to numbers and converts:"Not Available", "Too few to report", and "N/A" into NaN


In [25]:
numeric_cols = [
    'discharges',
    'readmissions',
    'excess_ratio',
    'predicted_rate',
    'expected_rate'
]

for col in numeric_cols:
    df_anon[col] = pd.to_numeric(df_anon[col], errors='coerce')


In [26]:
# To validate the above code:
df_anon[numeric_cols].isna().sum()

discharges        17
readmissions      17
excess_ratio      10
predicted_rate    10
expected_rate     10
dtype: int64

In [27]:
# Step 5: Aggregation of data 

In [28]:
# Define Analysis-Eligible Hospitals and validate
df_anon['analysis_eligible'] = df_anon['readmissions'].notna()

In [29]:
df_anon['analysis_eligible'].value_counts()

analysis_eligible
True     34
False    17
Name: count, dtype: int64

In [30]:
# Compute State of Colorado Benchmarks
#1: State Average Excess Readmission Ratio

state_avg_excess = (
    df_anon.loc[df_anon['analysis_eligible'], 'excess_ratio']
           .mean()
)


In [31]:
df_anon['state_avg_excess_ratio'] = state_avg_excess

In [32]:
#2: Compute Quartile Thresholds and assign performance_tiers
quartiles = (
    df_anon.loc[df_anon['analysis_eligible'], 'excess_ratio']
           .quantile([0.25, 0.5, 0.75])
)

q1, q2, q3 = quartiles


In [33]:
def performance_tier(x):
    if pd.isna(x):
        return 'Not Reported'
    elif x <= q1:
        return 'Top Quartile'
    elif x <= q2:
        return 'Above Average'
    elif x <= q3:
        return 'Below Average'
    else:
        return 'Bottom Quartile'

df_anon['performance_tier'] = df_anon['excess_ratio'].apply(performance_tier)


In [34]:
# Validate above function

df_anon['performance_tier'].value_counts()


performance_tier
Below Average      13
Above Average      10
Not Reported       10
Bottom Quartile     9
Top Quartile        9
Name: count, dtype: int64

In [35]:
# Derived Tableau Metrics to reduce calculated fields in Tableau
# Difference from State Average

df_anon['excess_vs_state_avg'] = (
    df_anon['excess_ratio'] - df_anon['state_avg_excess_ratio']
)

In [36]:
# Directional Flag

def performance_flag(x):
    if pd.isna(x):
        return 'Not Reported'
    elif x > 0:
        return 'Worse Than State Avg'
    else:
        return 'Better Than State Avg'

df_anon['performance_flag'] = df_anon['excess_vs_state_avg'].apply(performance_flag)


In [37]:
# Step 6: Final Data Validation: Entire dataframe is verified with: df_anon.info. This produced a multi-paged table. 
# I also used df_anon.head(), df_anon.tail() and df_anon.describe(). Everything looks good! 

In [38]:
# Step 7: Now proceeding to exporting the file to Tableau and to verify successful export
df_anon.to_csv(
    "colorado_heart_failure_project.csv",
    index=False
)


In [39]:
# Export Verification: I verified the .csv file was in the Jupyter file browser using the code below - which I commented out later for privacy.
# import os
# os.getcwd()

In [40]:
# Visual Confirmation: I opened my Desktop Tableau Public, connected to the .csv file and validated that data was correctly preserved.